In [89]:
!pip3 install tiktoken

In [90]:
import importlib
import tiktoken

print('tiktoken version:', importlib.metadata.version('tiktoken'))

tiktoken version: 0.9.0


In [91]:
tokenizer = tiktoken.get_encoding("gpt2")

In [92]:
text = ("Hello, do you like tea <|endoftext|> In the sunlit terraces" 
        "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special = {"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [93]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea <|endoftext|> In the sunlit terracesof someunknownPlace.


In [94]:
integers = tokenizer.encode('Akwirw ier')
print(integers)

strings = tokenizer.decode(integers)
print(strings)


[33901, 86, 343, 86, 220, 959]
Akwirw ier


In [95]:
#creating input-target- pairs
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

91275


In [96]:
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

91275


In [97]:
enc_sample = enc_text[50:]

In [98]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y: {y}")

x: [198, 220, 1366, 12]
y: [220, 1366, 12, 64]


In [99]:
for i in range(1, context_size +1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[198] ----> 220
[198, 220] ----> 1366
[198, 220, 1366] ----> 12
[198, 220, 1366, 12] ----> 64


In [100]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))


 ---->  

  ---->  data

  data ----> -

  data- ----> a


In [101]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special = {"|endoftext|>"})

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1: i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self,idx):
        return self.input_ids[idx], self.target_ids[idx]


In [105]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                        stride=128, shuffle=True, drop_last=True,
                        num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")

    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    dataloader = DataLoader(
        dataset,
        batch_size = batch_size,
        shuffle=shuffle,
        drop_last = drop_last,
        num_workers=num_workers,
    )

    return dataloader

In [107]:
with open('the-verdict.txt', "r", encoding="utf-8") as f:
    raw_text= f.read()

In [109]:
import torch
print("Pytorch version:", torch.__version__)
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)


Pytorch version: 2.7.1+cpu
[tensor([[628, 628, 198, 198]]), tensor([[628, 198, 198,  27]])]


In [111]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[628, 198, 198,  27]]), tensor([[198, 198,  27,   0]])]


In [113]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length = 4, stride=4, shuffle=False) 


data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[  628,   628,   198,   198],
        [   27,     0, 18227,  4177],
        [   56, 11401, 27711,    29],
        [  198,    27,  6494,   198],
        [  220, 42392,  2625,   268],
        [    1,   198,   220,   220],
        [  198,   220,  1366,    12],
        [ 8043,    12, 14171,  2625]])

Targets:
 tensor([[  628,   198,   198,    27],
        [    0, 18227,  4177,    56],
        [11401, 27711,    29,   198],
        [   27,  6494,   198,   220],
        [42392,  2625,   268,     1],
        [  198,   220,   220,   198],
        [  220,  1366,    12,  8043],
        [   12, 14171,  2625, 23736]])
